In [1]:
from torch.utils.data import DataLoader
from utils.dataset_TinyImage import load_data
from utils.OpenGAN_arc import Discriminator
from utils import train_openganII
from utils import visualize_results
from utils import evaluate_opengan
from utils import TinyImageClassifier
from utils.extr_fea import *
import warnings

warnings.filterwarnings('ignore')

In [2]:
config = {
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "batch_size": 128,
    "num_closed_classes": 200,  # TinyImageNet类别数
    "image_size": 64,  # 统一图像尺寸
    "feature_dim": 2048,  # ResNet101特征维度

    # 分类器训练参数
    "classifier_lr": 0.001,
    "classifier_epochs": 50,
    "classifier_weight_decay": 1e-3,

    # OpenGAN参数
    "opengan_lr": 1e-4,
    "opengan_epochs": 50,
    "lambda_g": 0.5,
    "d_train_ratio": 5,  # 判别器训练次数/生成器训练次数

    # 数据集路径
    "tinyimagenet_path": "./data/tiny-imagenet-200",
    "output_dir": "./results_setup2",
    "save_dir": "./results_setup2",

    # 开集数据集
    "open_datasets": ["CIFAR10", "SVHN", "MNIST"],
}

os.makedirs(config["output_dir"], exist_ok=True)

In [ ]:
if __name__ == '__main__':
    # 加载TinyImageNet数据集
    data_dict = load_data(config)
    closed_train_loader = DataLoader(
        data_dict["closed"]["train"],
        batch_size=config["batch_size"],
        shuffle=True
    )
    closed_val_loader = DataLoader(
        data_dict["closed"]["val"],
        batch_size=config["batch_size"],
        shuffle=False
    )
    # 训练闭集分类器
    classifier = train_classifierII(closed_train_loader, closed_val_loader,config)
    # 加载开集数据集
    open_train_data = data_dict["open"]["CIFAR10"]["train"]
    open_train_loader = DataLoader(
        open_train_data,
        batch_size=config["batch_size"],
        shuffle=True
    )
    val_data = {
    "closed": closed_val_loader,
    "open": DataLoader(
        data_dict["open"]["CIFAR10"]["test"], 
        batch_size=config["batch_size"],
        shuffle=False
    )
}
    # 训练OpenGAN
    discriminator = train_openganII(
        classifier, open_train_loader, val_data, config
    )
    # classifier = TinyImageClassifier(num_classes=config["num_closed_classes"]).to(config["device"])
    # classifier.load_state_dict(torch.load(os.path.join(config["save_dir"], "best_classifierII.pth")))
    best_discriminator = Discriminator(input_dim=2048).to(config["device"])
    best_discriminator.load_state_dict(
        torch.load(os.path.join(config["output_dir"], "best_discriminator.pth"))
    )
    # 最终测试评估
    test_auroc1 = evaluate_opengan(
        best_discriminator, classifier, data_dict["closed"]["test"], data_dict["open"]["CIFAR10"]["test"], config
    )
    test_auroc2 = evaluate_opengan(
        best_discriminator, classifier, data_dict["closed"]["test"], data_dict["open"]["SVHN"]["test"], config
    )
    test_auroc3 = evaluate_opengan(
        best_discriminator, classifier, data_dict["closed"]["test"], data_dict["open"]["MNIST"]["test"], config
    )
    print(f"\n最终测试集AUROC(CIFAR10): {test_auroc1:.4f}")
    print(f"\n最终测试集AUROC(SVHN): {test_auroc2:.4f}")
    print(f"\n最终测试集AUROC(MNIST): {test_auroc3:.4f}")

加载TinyImageNet数据集...
Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: ./data\train_32x32.mat
Using downloaded and verified file: ./data\test_32x32.mat

训练闭集分类器...


分类器 Epoch 1/50: 100%|██████████| 625/625 [14:32<00:00,  1.40s/it]


Epoch 1: 训练损失=4.1070, 验证准确率=0.1649


分类器 Epoch 2/50: 100%|██████████| 625/625 [06:45<00:00,  1.54it/s]


Epoch 2: 训练损失=3.4582, 验证准确率=0.2084


...

Epoch 48: D_loss=-8.0590, G_loss=16.1181, Val AUROC=0.5000


OpenGAN Epoch 49: 100%|██████████| 391/391 [00:57<00:00,  6.85it/s]


Epoch 49: D_loss=-8.0567, G_loss=16.1148, Val AUROC=0.5000


OpenGAN Epoch 50: 100%|██████████| 391/391 [00:56<00:00,  6.95it/s]


Epoch 50: D_loss=-8.0590, G_loss=16.1181, Val AUROC=0.5000

最终测试集AUROC(CIFAR10): 0.7040

最终测试集AUROC(SVHN): 0.8710

最终测试集AUROC(MNIST): 0.9736
